***
# TEST
***

In [ ]:
import os
import json
import pandas as pd
import pvdeg
from pytest import approx
from pvdeg import TEST_DATA_DIR

# Load weather data
WEATHER = pd.read_csv(
    os.path.join(TEST_DATA_DIR, "weather_day_pytest.csv"), index_col=0, parse_dates=True
)
with open(os.path.join(TEST_DATA_DIR, "meta.json"), "r") as file:
    META = json.load(file)

# Load expected results
rh_expected = pd.read_csv(
    os.path.join(TEST_DATA_DIR, "input_day_pytest.csv"), index_col=0, parse_dates=True
)
rh_cols = [col for col in rh_expected.columns if "RH" in col]
rh_expected = rh_expected[rh_cols]


def test_module():
    """
    test pvdeg.humidity.calc_rel_humidity

    Requires:
    ---------
    weather dataframe and meta dictionary
    """
    result = pvdeg.humidity.module(WEATHER, META)
    pd.testing.assert_frame_equal(result, rh_expected, check_dtype=False)


def test_psat():
    """
    test pvdeg.humidity.psat

    Requires:
    ---------
    weahter dataframe and meta dictionary
    """
    psat_avg = pvdeg.humidity.psat(temp=WEATHER["temp_air"])[1]
    assert psat_avg == approx(0.47607, abs=5e-5)

In [ ]:
test_psat()

In [ ]:
import pvdeg
import pickle
import pandas as pd
import numpy as np
import xarray as xr
import os
from pvdeg import TEST_DATA_DIR

GEO_META = pd.read_csv(os.path.join(TEST_DATA_DIR, "summit-meta.csv"), index_col=0)
with open(os.path.join(TEST_DATA_DIR, "summit-weather.pkl"), "rb") as f:
    GEO_WEATHER = pickle.load(f)

autotemplate

In [ ]:
autotemplate_result = pvdeg.geospatial.auto_template(
    func=pvdeg.humidity.module, ds_gids=GEO_WEATHER
).compute()

In [ ]:
humidity_template = xr.open_dataset(
    os.path.join(TEST_DATA_DIR, "humidity_template.nc")
).compute()

In [ ]:
def compare_datasets(ds1: xr.Dataset, ds2: xr.Dataset, atol=1e-10) -> bool:
    """Compare loaded datasets with "empty-like" values"""

    if ds1.dims != ds2.dims:
        return False

    if set(ds1.coords.keys()) != set(ds2.coords.keys()):
        return False

    for coord in ds1.coords:
        if ds1.coords[coord].dtype.kind in {"i", "f"}:
            # Use np.allclose for numeric coordinates
            if not np.allclose(ds1.coords[coord], ds2.coords[coord], atol=atol):
                return False
        elif ds1.coords[coord].dtype.kind == "M":  # datetime64 type
            # Use array equality for datetime coordinates
            if not np.array_equal(ds1.coords[coord], ds2.coords[coord]):
                return False
        else:
            if not np.array_equal(ds1.coords[coord], ds2.coords[coord]):
                return False

    if set(ds1.data_vars.keys()) != set(ds2.data_vars.keys()):
        return False

    for var in ds1.data_vars:
        if not np.allclose(ds1[var], ds2[var], atol=atol):
            return False

    for dim in ds1.dims:
        if not ds1.indexes[dim].equals(ds2.indexes[dim]):
            return False

    return True

In [ ]:
assert pvdeg.utilities.compare_datasets(autotemplate_result, humidity_template)

output template

In [ ]:
shapes = {
    "RH_surface_outside": ("gid", "time"),
    "RH_front_encap": ("gid", "time"),
    "RH_back_encap": ("gid", "time"),
    "RH_backsheet": ("gid", "time"),
}

manual_template = pvdeg.geospatial.output_template(
    shapes=shapes, ds_gids=GEO_WEATHER
).compute()

In [ ]:
pvdeg.utilities.compare_datasets(manual_template, humidity_template)

In [ ]:
# test template

shapes = {"testA": ("gid",), "testB": ("gid", "time")}

template = pvdeg.geospatial.output_template(
    shapes=shapes,
    ds_gids=GEO_WEATHER,
)

In [ ]:
template.to_netcdf(os.path.join(TEST_DATA_DIR, "mismatch-template.nc"))

In [1]:
import pvdeg
from pvdeg import TEST_DATA_DIR
import pickle
import pandas as pd
import numpy as np
import xarray as xr
import os

GEO_META = pd.read_csv(os.path.join(TEST_DATA_DIR, "summit-meta.csv"), index_col=0)

with open(os.path.join(TEST_DATA_DIR, "summit-weather.pkl"), "rb") as f:
    GEO_WEATHER = pickle.load(f).compute().load()

HUMIDITY_TEMPLATE = xr.open_dataset(
    os.path.join(TEST_DATA_DIR, "humidity_template.nc"), engine="h5netcdf"
).compute()

In [ ]:
GEO_WEATHER

In [ ]:
GEO_WEATHER.chunks

In [ ]:
HUMIDITY_TEMPLATE.chunks

In [14]:
shapes = {
    "RH_surface_outside": ("gid", "time"),
    "RH_front_encap": ("gid", "time"),
    "RH_back_encap": ("gid", "time"),
    "RH_backsheet": ("gid", "time"),
}

# falsely assigning chunks here
manual_template = pvdeg.geospatial.output_template(shapes=shapes, ds_gids=GEO_WEATHER)

assert pvdeg.utilities.compare_templates(manual_template, HUMIDITY_TEMPLATE)
for k, v in manual_template.chunks.items():
    if len(v) != 1:
        raise ValueError(f"""
                          Need one chunk per axis for an unchunked input
                          dimension {k} has {len(v)} chunks.
                          """)

1
1


In [17]:
chunked_weather = GEO_WEATHER.chunk({"gid": 3})

In [22]:
HUMIDITY_TEMPLATE.chunk({"gid": 3})

<xarray.Dataset> Size: 6MB
Dimensions:             (gid: 11, time: 17520)
Coordinates:
  * gid                 (gid) int64 88B 449211 452064 453020 ... 460613 462498
  * time                (time) datetime64[ns] 140kB 2022-01-01 ... 2022-12-31...
Data variables:
    RH_surface_outside  (gid, time) float64 2MB dask.array<chunksize=(3, 17520), meta=np.ndarray>
    RH_front_encap      (gid, time) float64 2MB dask.array<chunksize=(3, 17520), meta=np.ndarray>
    RH_back_encap       (gid, time) float64 2MB dask.array<chunksize=(3, 17520), meta=np.ndarray>
    RH_backsheet        (gid, time) float64 2MB dask.array<chunksize=(3, 17520), meta=np.ndarray>

In [23]:
chunked_template

<xarray.Dataset> Size: 352B
Dimensions:  (gid: 11)
Coordinates:
  * gid      (gid) int64 88B 449211 452064 453020 ... 459670 460613 462498
Data variables:
    x        (gid) float64 88B dask.array<chunksize=(3,), meta=np.ndarray>
    T98_0    (gid) float64 88B dask.array<chunksize=(3,), meta=np.ndarray>
    T98_inf  (gid) float64 88B dask.array<chunksize=(3,), meta=np.ndarray>

In [29]:
pvdeg.utilities.compare_templates(chunked_template, HUMIDITY_TEMPLATE.chunk({"gid": 3}))

C:\Users\tford\dev\PVDegradationTools\pvdeg\utilities.py:1231: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  if ds1.dims != ds2.dims:


True

In [28]:
chunked_template = pvdeg.geospatial.auto_template(
    ds_gids=chunked_weather, func=pvdeg.humidity.module
)

geo_res = pvdeg.geospatial.analysis(
    weather_ds=chunked_weather,
    meta_df=GEO_META,
    func=pvdeg.humidity.module,
    template=chunked_template,
)

The array tilt angle was not provided, therefore the latitude tilt of 39.9 was used.
The array tilt angle was not provided, therefore the latitude tilt of 39.5 was used.
The array tilt angle was not provided, therefore the latitude tilt of 39.5 was used.
The array tilt angle was not provided, therefore the latitude tilt of 39.8 was used.
The array tilt angle was not provided, therefore the latitude tilt of 39.4 was used.
The array tilt angle was not provided, therefore the latitude tilt of 39.8 was used.
The array tilt angle was not provided, therefore the latitude tilt of 39.6 was used.
The array tilt angle was not provided, therefore the latitude tilt of 39.9 was used.
The array tilt angle was not provided, therefore the latitude tilt of 39.6 was used.
The array tilt angle was not provided, therefore the latitude tilt of 39.4 was used.
The array tilt angle was not provided, therefore the latitude tilt of 39.7 was used.


In [ ]:
geo_res = pvdeg.geospatial.analysis(
    chunked_weather,
    meta_df=GEO_META,
    func=pvdeg.humidity.module,
)

In [ ]:
res_ds = pvdeg.geospatial.analysis(
    weather_ds=GEO_WEATHER,
    meta_df=GEO_META,
    func=pvdeg.standards.standoff,
)

data_var = res_ds["x"]

# Stack the latitude and longitude coordinates into a single dimension
# convert to dataframe, this can be done with xr.dataset.to_dataframe as well
stacked = data_var.stack(z=("latitude", "longitude"))
latitudes = stacked["latitude"].values
longitudes = stacked["longitude"].values
data_values = stacked.values
combined_array = np.column_stack((latitudes, longitudes, data_values))

res = pd.DataFrame(combined_array).dropna()
ans = pd.read_csv(os.path.join(TEST_DATA_DIR, "summit-standoff-res.csv"), index_col=0)
res.columns = ans.columns

# pd.testing.assert_frame_equal(res, ans, check_dtype=False, check_names=False)

In [ ]:
res_ds.chunks